In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import seaborn as sns
import pandas as pd

In [ ]:
hero_df = pd.read_csv('hero-network.csv')
edges_df = pd.read_csv('hero-comic.csv')
nodes_df = pd.read_csv('nodes.csv')

In [ ]:
# Deletes the '/' character at the end of the of each name and truncates the words to 20 characters
hero_df['hero1'] = hero_df['hero1'].apply(lambda x: x.rstrip('/')[:20])
hero_df['hero2'] = hero_df['hero2'].apply(lambda x: x.rstrip('/')[:20])
edges_df['hero'] = edges_df['hero'].apply(lambda x: x.rstrip('/')[:20])
nodes_df['node'] = nodes_df['node'].apply(lambda x: x.rstrip('/')[:20])

#Deletes lines with a hero linked to itselfhero_df.drop(hero_df[hero_df['hero1'] == hero_df['hero2']].index, inplace = True)
hero_df.drop(hero_df[hero_df['hero1'] == hero_df['hero2']].index, inplace = True)

## Hero-Comic

Takes the graph with hero-comic pairs. 

1. Number of nodes and edges
2. Plot the graph with 'fruchterman_reingold_layout'
3. Plot the graph with Degree Centrality and Betweenness Centrality
 

In [ ]:
graph = nx.Graph()
graph.add_edges_from(edges_df[["hero", "comic"]].values)
pos = nx.fruchterman_reingold_layout(graph)

In [ ]:
print("NODES: ", len(graph.nodes))
print("EDGES: ", len(graph.edges))

In [ ]:
plt.figure(figsize=(20,12))
nx.draw(graph, pos)
plt.show()

In [ ]:
deg_centrality = nx.degree_centrality(graph)

cent = np.fromiter(deg_centrality.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(graph, pos, node_size=sizes, node_color=sizes, cmap=colormap)
plt.show()

In [ ]:
bet = nx.betweenness_centrality(graph)
plt.figure(figsize=(15,10))
sns.distplot(list(bet.values()), kde=False)

In [ ]:
cent = np.fromiter(bet.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(graph, pos, node_size=sizes, node_color=sizes, cmap=colormap)
plt.show()

### Connected components

5. The largest connected component is taken
6. Prints the number of nodes and nodes and nodes of the largest connected component

In [ ]:
components = list(nx.connected_components(graph))
largest_component = max(components, key=len)
H = graph.subgraph(largest_component)

In [ ]:
print("NODES: ", len(H.nodes))
print("EDGES: ", len(H.edges))

#### Reducing the number of nodes

I reduce the number of nodes as the graph is too large to perform certain analyses, 1000 nodes are taken

In [ ]:
from collections import deque

def connectedSubgraph(graph):
    # Initialize an empty graph object
    subgraph = nx.Graph()
    # Add the first node in the input graph to the subgraph
    subgraph.add_node(next(iter(graph)))
    
    # Create a queue and add the first node of the input graph to it
    queue = deque([next(iter(graph))])
    
    # While the queue is not empty and the number of nodes in the subgraph is less than 1000
    while queue and subgraph.number_of_nodes() < 1000:
        # Remove the first node from the queue
        node = queue.popleft()
        
        # Iterate through the adjacent nodes of the current node
        for adjacent in graph[node]:
            # If the adjacent node is not in the subgraph
            if adjacent not in subgraph:
                # Add the adjacent node to the subgraph
                subgraph.add_node(adjacent)
                # Add an edge between the current node and the adjacent node in the subgraph
                subgraph.add_edge(node, adjacent)
                # Add the adjacent node to the queue
                queue.append(adjacent)
  
    # return the subgraph
    return subgraph

subgraph = connectedSubgraph(H)


The same analyses as above are performed by adding different layouts, calculation of the 10 densest nodes, and calculation of eccentricity, diameter, radius, periphery, and center values. 

In [ ]:
print("NODES: ", len(subgraph.nodes))
print("EDGES: ", len(subgraph.edges))

In [ ]:
posSpring = nx.spring_layout(subgraph)
plt.figure(figsize=(20,12))
nx.draw(subgraph, posSpring)
plt.show()

In [ ]:
posPlanar = nx.planar_layout(subgraph)
plt.figure(figsize=(20,12))
nx.draw(subgraph, posPlanar )
plt.show()

In [ ]:
posKamada = nx.kamada_kawai_layout(subgraph)
plt.figure(figsize=(20,12))
nx.draw(subgraph, posKamada)
plt.show()

In [ ]:
posSpiral = nx.spiral_layout(subgraph)
plt.figure(figsize=(20,12))
nx.draw(subgraph, posSpiral)
plt.show()

In [ ]:
deg_centrality = nx.degree_centrality(subgraph)

cent = np.fromiter(deg_centrality.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(subgraph, posSpring, node_size=sizes, node_color=sizes, cmap=colormap)
plt.show()

In [ ]:
deg_centrality = nx.degree_centrality(subgraph)

cent = np.fromiter(deg_centrality.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(subgraph, posKamada, node_size=sizes, node_color=sizes, cmap=colormap)
plt.show()

In [ ]:
bet = nx.betweenness_centrality(graph)
plt.figure(figsize=(15,10))
sns.distplot(list(bet.values()), kde=False)

In [ ]:
cent = np.fromiter(bet.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(graph, pos, node_size=sizes, node_color=sizes, cmap=colormap)
plt.show()

In [ ]:
degrees = subgraph.degree()
sorted_degrees = sorted(degrees, key=lambda x: x[1], reverse=True)

print("I 10 nodi più densi sono:")
for node, degree in sorted_degrees[:10]:
    print(f"  {node}: grado {degree}")

In [ ]:
eccentricity = nx.eccentricity(subgraph)
diameter = nx.diameter(subgraph)
radius = nx.radius(subgraph)
periphery = [node for node, ecc in eccentricity.items() if ecc == diameter]
center = [node for node, ecc in eccentricity.items() if ecc == radius]

print("ECCENTRICITY: ",len(eccentricity))
print("DIAMETER: ",diameter)
print("RADIUS: ",radius)
print("PERIPHERY: ",len(periphery))
print("CENTER: ",len(center))

## Hero-Hero Network

Same analysis as above by adding using the graph generated with the connections between heroes.

In [ ]:
graph = nx.Graph()
graph.add_edges_from(hero_df[["hero1", "hero2"]].values)


print("NODES: ", len(graph.nodes)) 
print("EDGES: ", len(graph.edges))

In [ ]:
components = list(nx.connected_components(graph))
largest_component = max(components, key=len)
H = graph.subgraph(largest_component)

In [ ]:
pos = nx.spring_layout(graph)

plt.figure(figsize=(20,12))
nx.draw(graph, pos)
plt.show()

In [ ]:
pos = nx.spiral_layout(graph)

plt.figure(figsize=(20,12))
nx.draw(graph, pos)
plt.show()

In [ ]:
pos = nx.fruchterman_reingold_layout(graph)

plt.figure(figsize=(20,12))
nx.draw(graph, pos)
plt.show()

In [ ]:
deg_centrality = nx.degree_centrality(graph)

cent = np.fromiter(deg_centrality.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(graph, pos, node_size=sizes, node_color=sizes, cmap=colormap) #LAYOUT SPRING
plt.show()

In [ ]:
bet = nx.betweenness_centrality(graph)
plt.figure(figsize=(15,10))
sns.distplot(list(bet.values()), kde=False)

In [ ]:
cent = np.fromiter(bet.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(graph, pos, node_size=sizes, node_color=sizes, cmap=colormap)
plt.show()

### Weighted Graph

Analysis is performed on the graph by adding weights. 
Degree assortativity coefficient is calculated.

In [ ]:
# Create a new dataframe 'weights' by grouping the rows of 'hero_df' by columns and counting the size of each
# i.e., the number of appearances together 
weights = hero_df.groupby(hero_df.columns.tolist()).size().reset_index().rename(columns={0: 'weight'})

# Group the data by hero1 and hero2 and sum the total weight 
weights[['hero1','hero2']] = pd.DataFrame(np.sort(weights[['hero1', 'hero2']].values, axis=1))
hero_weighted = weights.groupby(['hero1', 'hero2']).weight.sum().reset_index()

# Calculate the inverse weight
hero_weighted['weight'] = round(1/hero_weighted['weight'],1)

# Create a multi graph from the hero_weighted dataframe
graphWeight = nx.from_pandas_edgelist(hero_weighted,'hero1','hero2', edge_attr='weight', create_using=nx.MultiGraph())


In [ ]:
components = list(nx.connected_components(graphWeight))
largest_component = max(components, key=len)
graph_weighted = graphWeight.subgraph(largest_component)

In [ ]:
springWeight = nx.spring_layout(H)
plt.figure(figsize=(20,12))
nx.draw(graph_weighted, springWeight )
plt.show()

In [ ]:
kamadaWeight = nx.kamada_kawai_layout(H)
plt.figure(figsize=(20,12))
nx.draw(graph_weighted, kamadaWeight )
plt.show()

In [ ]:
spiralWeight = nx.spiral_layout(H)
plt.figure(figsize=(20,12))
nx.draw(graph_weighted, spiralWeight )
plt.show()

In [ ]:
deg_centrality = nx.degree_centrality(H)

cent = np.fromiter(deg_centrality.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(graph_weighted, kamadaWeight, node_size=sizes, node_color=sizes, cmap=colormap)
plt.show()

In [ ]:
betWeight = nx.betweenness_centrality(H)
plt.figure(figsize=(15,10))
sns.distplot(list(bet.values()), kde=False)

In [ ]:

cent = np.fromiter(betWeight.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)

plt.colorbar(scalarmappaple)
nx.draw(graph_weighted, kamadaWeight, node_size=sizes, node_color=sizes, cmap=colormap)
plt.show()

In [ ]:
eccentricity = nx.eccentricity(graph_weighted)
diameter = nx.diameter(graph_weighted)
radius = nx.radius(graph_weighted)
periphery = [node for node, ecc in eccentricity.items() if ecc == diameter]
center = [node for node, ecc in eccentricity.items() if ecc == radius]
assortativity = nx.degree_assortativity_coefficient(graph_weighted)

In [ ]:
n_nodes = graph_weighted.number_of_nodes()
n_edges = graph_weighted.number_of_edges()
print("NODES: " + n_nodes)
print("EDGES: " + n_edges)

In [ ]:
density = 2*n_edges/(n_nodes*(n_nodes-1))
print("DENSITY: " + density)

In [ ]:
print("ASSORTATIVITY: " + assortativity)
print("RADIUS: " + radius)
print("DIAMETER: " + diameter)
print("NODES IN PERIPHERY: " + len(periphery))
print("NODES IN CENTER: " + len(center))

## Ego Network - Iron Man

Let's create an Ego Network on the character of Iron Man. On this Ego Network we go to calculate eigenvector_centrality, closeness and pagerank.

In [ ]:
ego_graph = nx.ego_graph(graph, 'IRON MAN/TONY STARK ')
ego_graph = nx.Graph(ego_graph)

In [ ]:
print("NUMERO DI NODI EGO NETWORK IRON MAN:", ego_graph.number_of_nodes())
print("NUMERO DI ARCHI EGO NETWORK IRON MAN:", ego_graph.number_of_edges())
print("GRAFO PESATO: ", nx.is_weighted(ego_graph))  # Output: True

In [ ]:
ego_pos = nx.spring_layout(ego_graph)
plt.figure(figsize=(20, 20))
nx.draw(ego_graph, ego_pos)
plt.show()

In [ ]:
closeness_centrality = nx.closeness_centrality(ego_graph)

plt.figure(figsize=(20,15))
cent = np.fromiter(closeness_centrality.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)
plt.colorbar(scalarmappaple)

nx.draw(ego_graph, ego_pos, node_size=25, node_color=sizes, cmap=colormap) #LAYOUT SPRING
plt.show()

In [ ]:
eigenvector_centrality = nx.eigenvector_centrality(ego_graph)

plt.figure(figsize=(20,15))
cent = np.fromiter(eigenvector_centrality.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)
plt.colorbar(scalarmappaple)

nx.draw(ego_graph, ego_pos, node_size=25, node_color=sizes, cmap=colormap) #LAYOUT SPRING
plt.show()

In [ ]:
pagerank = nx.pagerank(ego_graph)

plt.figure(figsize=(20,15))
cent = np.fromiter(pagerank.values(), float)
sizes = cent / np.max(cent) * 200
normalize = mcolors.Normalize(vmin=cent.min(), vmax=cent.max())
colormap = plt.cm.viridis

scalarmappaple = plt.cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(cent)
plt.colorbar(scalarmappaple)

nx.draw(ego_graph, ego_pos, node_size=25, node_color=sizes, cmap=colormap) #LAYOUT SPRING
plt.show()

### Communities


Analysis on network partitions and creation of a micro-network with only the best edges.

In [ ]:
partition = community_louvain.best_partition(ego_graph)  # compute communities

plt.figure(figsize=(20,20)) 
plt.axis('off')
nx.draw_networkx_nodes(ego_graph, ego_pos, node_size=60, cmap=plt.cm.RdYlBu, node_color=list(partition.values()))
nx.draw_networkx_edges(ego_graph, ego_pos, alpha=0.3)
plt.show(ego_graph)

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

community_count = max(partition.values()) +1

community_similarity = np.zeros((community_count, community_count))
for node1, com1 in partition.items():
    for node2, com2 in partition.items():
        if com1 == com2:
            continue
        if ego_graph.has_edge(node1, node2):
            community_similarity[com1][com2] += 1

# perform linkage clustering on the similarity matrix
linkage_matrix = linkage(community_similarity, method='ward')

# plot dendrogram
plt.figure(figsize=(20,10))
dendrogram(linkage_matrix, labels=[f"Community {i}" for i in range(community_count)])
plt.show()

In [ ]:
weight_threshold = 0.01
# Itera on graph nodes
for node in list(ego_graph.nodes()):
  # Itera non node neighbors
  for neighbor in list(ego_graph.neighbors(node)):
    # If the weight of the edge is above the threshold, remove the node and edge
    if ego_graph[node][neighbor]["weight"] > weight_threshold:
      ego_graph.remove_edge(node, neighbor)

for node in list(ego_graph.nodes()):
  # If the node has no neighbors, remove it
  if len(list(ego_graph.neighbors(node))) ==0:
    ego_graph.remove_node(node)

components = list(nx.connected_components(ego_graph))
largest_component = max(components, key=len)
ego_graph = ego_graph.subgraph(largest_component)

print("NODES:", ego_graph.number_of_nodes())
print("EDGES:", ego_graph.number_of_edges())
print("WEIGHTED:" + nx.is_weighted(graph))  # Output: True

In [ ]:
ego_pos = nx.spring_layout(ego_graph)
plt.figure(figsize=(20, 20))
nx.draw(ego_graph, ego_pos, node_color= '#ff0000', node_size = 125, with_labels = True)
plt.show()